In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Datasets/PlantVillage/archive (2).zip"
extract_path = "/content/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed!")

Extraction completed!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

input_folder = "/content/PlantVillage"
output_folder = "/content/PlantVillage_Split"

splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(.8, .1, .1)   # train, val, test
)

Copying files: 20639 files [00:04, 4954.57 files/s]


In [ ]:
train_dir="/content/PlantVillage_Split/train"
valid_dir="/content/PlantVillage_Split/val"
test_dir="/content/PlantVillage_Split/test"
img=256
b_s=32

In [ ]:
train_datagen= ImageDataGenerator(
    rescale=1/255.,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
valid_test_datagen=ImageDataGenerator(rescale=1/255.)

In [ ]:
train=train_datagen.flow_from_directory(
    train_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=True,
    seed=123
)

Found 16504 images belonging to 15 classes.


In [ ]:
train[0]

(array([[[[0.7348758 , 0.73095423, 0.7858562 ],
          [0.7457799 , 0.7418583 , 0.79676026],
          [0.75906706, 0.7551455 , 0.81004745],
          ...,
          [0.6627451 , 0.654902  , 0.7058824 ],
          [0.67939025, 0.6715471 , 0.7225275 ],
          [0.6736791 , 0.665836  , 0.71681637]],
 
         [[0.7499366 , 0.746015  , 0.80091697],
          [0.75082725, 0.7469057 , 0.80180764],
          [0.7501231 , 0.7462015 , 0.8011035 ],
          ...,
          [0.6627451 , 0.654902  , 0.7058824 ],
          [0.68060756, 0.6727644 , 0.7237448 ],
          [0.67266464, 0.6648215 , 0.71580195]],
 
         [[0.75982946, 0.7559079 , 0.81080985],
          [0.75421417, 0.7502926 , 0.80519456],
          [0.74605274, 0.7421312 , 0.79703313],
          ...,
          [0.6627451 , 0.654902  , 0.7058824 ],
          [0.68182486, 0.6739817 , 0.7249621 ],
          [0.6716503 , 0.66380715, 0.71478754]],
 
         ...,
 
         [[0.57853955, 0.56819266, 0.6204249 ],
          [0.60068

In [ ]:
valid=valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=False
)

Found 2058 images belonging to 15 classes.


In [ ]:
test=valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=False
)

Found 2076 images belonging to 15 classes.


In [ ]:
classes=train.num_classes
classes

15

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

In [ ]:
m=keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,activation="relu",input_shape=[256,256,3]),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=128,kernel_size=3,activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128,activation="relu"),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(units=256,activation="relu"),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(units=128,activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units=15,activation="softmax"),

])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
m.compile(optimizer=Adam(learning_rate=0.001),loss="categorical_crossentropy",metrics=["accuracy"])
m.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,906,831 (56.87 MB)

 Trainable params: 14,906,831 (56.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint

In [ ]:
check_path=os.path.join('/content/drive/MyDrive/Colab Notebooks/checkpoints', 'my_model_checkpoint.keras')

In [ ]:
cb=[
    EarlyStopping(monitor="val_loss",patience=7,restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss",factor=0.2, patience=3),
    ModelCheckpoint(filepath=check_path,save_best_only=True)
]

In [ ]:
history=m.fit(train,epochs=50,validation_data=valid,callbacks=cb)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 281s 523ms/step - accuracy: 0.2071 - loss: 2.4281 - val_accuracy: 0.4898 - val_loss: 1.8431 - learning_rate: 0.0010
Epoch 2/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 258s 500ms/step - accuracy: 0.5120 - loss: 1.5267 - val_accuracy: 0.6817 - val_loss: 0.9731 - learning_rate: 0.0010
Epoch 3/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 259s 501ms/step - accuracy: 0.6302 - loss: 1.1162 - val_accuracy: 0.6939 - val_loss: 0.9282 - learning_rate: 0.0010
Epoch 4/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 259s 503ms/step - accuracy: 0.6761 - loss: 0.9685 - val_accuracy: 0.6812 - val_loss: 1.1149 - learning_rate: 0.0010
Epoch 5/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 256s 497ms/step - accuracy: 0.7205 - loss: 0.8535 - val_accuracy: 0.6783 - val_loss: 1.2069 - learning_rate: 0.0010
Epoch 6/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 258s 499ms/step - accuracy: 0.7559 - loss: 0.7396 - val_accuracy: 0.8003 - val_loss: 0.6234 - learning_rate: 0.0010
Epoch 7/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 260s 504ms/step - accura